In [ ]:
import pandas as pd
import numpy as np
import os
import cv2
from PIL import Image, ImageDraw
from ast import literal_eval
import matplotlib.pyplot as plt
import urllib
from tqdm.notebook import tqdm



In [ ]:
# Install Retinanet

!git clone https://github.com/fizyr/keras-retinanet


In [ ]:
%cd keras-retinanet/

!pip install .
!python setup.py build_ext --inplace

In [ ]:
import numpy as np
import shutil
import pandas as pd
import os, sys, random
import xml.etree.ElementTree as ET
import pandas as pd
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
from PIL import Image
import requests
import urllib
from keras_retinanet.utils.visualization import draw_box, draw_caption , label_color

from keras_retinanet.utils.image import preprocess_image, resize_image

import tensorflow as tf
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
from keras_retinanet.utils.compute_overlap import compute_overlap

In [ ]:
import pandas as pd



pngPath='/kaggle/input/uftp-covid-lung/Kaggle_LabelImg/treinoTodas/'
annotPath='/kaggle/input/uftp-covid-lung/Kaggle_LabelImg/treinoTodas/'

# ../input/uftp-covid-lung/Kaggle_LabelImg/validacaoTodas
pngPath_val='/kaggle/input/uftp-covid-lung/Kaggle_LabelImg/validacaoTodas/'
annotPath_val='/kaggle/input/uftp-covid-lung/Kaggle_LabelImg/validacaoTodas/'


data=pd.DataFrame(columns=['fileName','xmin','ymin','xmax','ymax','class'])
data_val=pd.DataFrame(columns=['fileName','xmin','ymin','xmax','ymax','class'])

os.getcwd()
#read All files
allfiles = [f for f in listdir(annotPath) if isfile(join(annotPath, f))]
allfiles_val = [f for f in listdir(annotPath_val) if isfile(join(annotPath_val, f))]

print('Train files:'+ str(len(allfiles)))
print('Val files:'+ str(len(allfiles_val)))

totalNormal = 0
totalCovid = 0

for file in allfiles:
  if "Normal" in file:
    totalNormal += 1
  if "COVID" in file:
    totalCovid += 1

print('total normal: ' + str(totalNormal))
print('total covid: ' + str(totalCovid))

In [ ]:

#Read all pdf files in images and then in text and store that in temp folder 
#Read all pdf files in images and then in text and store that in temp folder
for file in allfiles:
  #print(file)
  if (file.split(".")[1]=='xml'):
    fileName='/kaggle/input/uftp-covid-lung/Kaggle_LabelImg/treinoTodas/'+file.replace(".xml",'.png')
    tree = ET.parse(annotPath+file)
    root = tree.getroot()
    for obj in root.iter('object'):
        cls_name = obj.find('name').text
        xml_box = obj.find('bndbox')
        xmin = xml_box.find('xmin').text
        ymin = xml_box.find('ymin').text
        xmax = xml_box.find('xmax').text
        ymax = xml_box.find('ymax').text
        # Append rows in Empty Dataframe by adding dictionaries
        
        data = data.append({'fileName': fileName, 'xmin': xmin, 'ymin':ymin,'xmax':xmax,'ymax':ymax,'class':cls_name}, ignore_index=True)

data.shape


#Read all pdf files in images and then in text and store that in temp folder 
#Read all pdf files in images and then in text and store that in temp folder
for file in allfiles_val:
  #print(file)
  if (file.split(".")[1]=='xml'):
    fileName='/kaggle/input/uftp-covid-lung/Kaggle_LabelImg/validacaoTodas/'+file.replace(".xml",'.png')
    tree = ET.parse(annotPath_val+file)
    root = tree.getroot()
    for obj in root.iter('object'):
        cls_name = obj.find('name').text
        xml_box = obj.find('bndbox')
        xmin = xml_box.find('xmin').text
        ymin = xml_box.find('ymin').text
        xmax = xml_box.find('xmax').text
        ymax = xml_box.find('ymax').text
        # Append rows in Empty Dataframe by adding dictionaries
        
        data_val = data_val.append({'fileName': fileName, 'xmin': xmin, 'ymin':ymin,'xmax':xmax,'ymax':ymax,'class':cls_name}, ignore_index=True)

data_val.shape

In [ ]:
# data.to_csv('/kaggle/input/lungcovid/ufp/Kaggle_LabelImg/treinoTodas/annotationFormat.csv')
print(data.head)

In [ ]:
# pick a random image
filepath = data.sample()['fileName'].values[0]

# get all rows for this image
df2 = data[data['fileName'] == filepath]
im = np.array(Image.open(filepath))

# if there's a PNG it will have alpha channel
#im = im[:,:,:3]

for idx, row in df2.iterrows():
    box = [
      row['xmin'],
      row['ymin'],
      row['xmax'],
      row['ymax'],
    ]
    print(box)
    draw_box(im, box, color=(255, 0, 0))

plt.axis('off')
plt.imshow(im)
plt.show()                        

#show_image_with_boxes(data)

In [ ]:
#Check few records of data
data.head()

In [ ]:
data.to_csv('../annotation.csv', header = False, index=False)
data_val.to_csv('../annotation_val.csv', header = False, index=False)

In [ ]:
#Define labels & write them in a file
classes = ['Normal','C19']
with open('../maskDetectorClasses.csv', 'w') as f:
  for i, class_name in enumerate(classes):
    f.write(f'{class_name},{i}\n')         

if not os.path.exists('snapshots'):
  os.mkdir('snapshots')

In [ ]:
PRETRAINED_MODEL = './snapshots/_pretrained_model.h5'

URL_MODEL = 'https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5'
urllib.request.urlretrieve(URL_MODEL, PRETRAINED_MODEL)

print('Downloaded pretrained model to ' + PRETRAINED_MODEL)

In [ ]:
from tensorflow.keras import backend

#tf.compat.v1.Session()
#python keras_retinanet/bin/train.py --gpu 0 --snapshot-path ./snapshots --backbone resnet50 --batch-size 1 --epochs 2000 --steps 1000 --tensorboard-dir=./log --weights ~/.keras/models/ResNet-50-model.keras.h5 csv images.train.csv classes.csv
# total of 4894 images

#Put your training data path & file that has labels for your training data
!keras_retinanet/bin/train.py --freeze-backbone  --random-transform --snapshot-path ./snapshots  --weights {PRETRAINED_MODEL}  --steps 250 --tensorboard-dir=./log --epochs 100 csv ../annotation.csv ../maskDetectorClasses.csv

In [ ]:
from glob import glob
model_paths = glob('./snapshots/resnet50_csv_0*.h5')
#model_paths = glob('snapshots/resnet50_csv_15.h5')
latest_path = sorted(model_paths)[-1]
print("path:", latest_path)


from keras_retinanet import models

model = models.load_model(latest_path, backbone_name='resnet50')
model = models.convert_model(model)
model.save('../modelo_11_06.h5')

label_map = {}
for line in open('../maskDetectorClasses.csv'):
  row = line.rstrip().split(',')
  label_map[int(row[1])] = row[0]

In [ ]:
# /kaggle/input/uftp-covid-lung/Kaggle_LabelImg/validacaoTodas/annotation.csv

# python ./bin/convert_model.py ./snapshots/xx.h5 ./infer_models/xx.h5

!keras_retinanet/bin/evaluate.py csv ../annotation_val.csv  ../maskDetectorClasses.csv  ../modelo_11_06.h5


In [ ]:
# os.chdir(r'/kaggle/working')
# %cd /kaggle/working/keras-retinanet/snapshots
# %ls

from IPython.display import FileLink
# FileLink(r'resnet50_csv_30.h5')
FileLink(r'/kaggle/working/keras-retinanet/snapshots/resnet50_csv_50.h5')

In [ ]:
model.summary()

In [ ]:
import time

def img_retinaNet_test():
  # choose a random image
  row = data.sample()
  filepath = row['fileName'].values[0]
  print("filepath:", filepath)
  # get all rows for this image
  df2 = data[data['fileName'] == filepath]
  im = np.array(Image.open(filepath))
  print("im.shape:", im.shape)

  #plt.imshow(im)


  # copy to draw on
  draw = im.copy()
  draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

  # preprocess image for network
  image = preprocess_image(draw)
  image, scale = resize_image(image)

  # process image
  start = time.time()
  boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
  print("processing time: ", time.time() - start)


  # correct for image scale
  boxes /= scale

  # visualize detections
  for box, score, label in zip(boxes[0], scores[0], labels[0]):
      # scores are sorted so we can break
      if score < 0.5:
          break
          
      color = label_color(label)
      
      b = box.astype(int)
      draw_box(draw, b, color=color)
      
      caption = "{} {:.3f}".format(classes[label], score)
      draw_caption(draw, b, caption)
      
  plt.figure(figsize=(15, 15))
  plt.axis('off')
  plt.imshow(draw)
  plt.show()



In [ ]:
img_retinaNet_test()

img_retinaNet_test()

img_retinaNet_test()

img_retinaNet_test()

img_retinaNet_test()

img_retinaNet_test()

In [ ]:
fig, ax = plt.subplots(nrows=3, ncols=3, figsize=(10, 10))
for i in range(3):
     for j in range(3):
      row = data.sample()
      filepath = row['fileName'].values[0]
      print("filepath:", filepath)
      # get all rows for this image
      df2 = data[data['fileName'] == filepath]
      im = np.array(Image.open(filepath))
      print("im.shape:", im.shape)

       # copy to draw on
      draw = im.copy()
      draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

      # preprocess image for network
      image = preprocess_image(draw)
      image, scale = resize_image(image)

      # process image
      start = time.time()
      boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
      print("processing time: ", time.time() - start)

      real = filepath.split('/')[6]

      # correct for image scale
      boxes /= scale

      # visualize detections
      for box, score, label in zip(boxes[0], scores[0], labels[0]):
          # scores are sorted so we can break
          if score < 0.5:
              break
              
          color = label_color(label)
          
          b = box.astype(int)
          draw_box(draw, b, color=color)
          
          caption = "{} {:.3f}".format('predito: '+ classes[label] + ' Correto:' + real, score)
          draw_caption(draw, b, caption)
          
      
      ax[i][j].axis('off')
      ax[i][j].imshow(draw)
  
        

plt.show()

In [ ]:


y_val = list()
predictions = list()
os.getcwd()
  

In [ ]:

#read All files
allfilesCovid = [f for f in listdir(pngPath_val) if isfile(join(pngPath_val, f))]
# print(allfilesCovid)
print(len(allfilesCovid))

for file in allfilesCovid:
    if (file.split(".")[1]=='png'):
#         y_val.append(1)
        im = np.array(Image.open(pngPath_val + file))
         # copy to draw on
        draw = im.copy()
        draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

        # preprocess image for network
        image = preprocess_image(draw)
        image, scale = resize_image(image)


        boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
    #     print("processing time: ", time.time() - start)

        # correct for image scale
        boxes /= scale

        # visualize detections
        for box, score, label in zip(boxes[0], scores[0], labels[0]):
            if label == 1:
                predictions.append(1)
                if 'COVID' in str(file):
                    y_val.append(1)
                else:
                   y_val.append(0)
            if label == 0:
                predictions.append(0)
                if 'COVID' in str(file):
                    y_val.append(1)
                else:
                   y_val.append(0)
            break

        
print(len(predictions))

In [ ]:

print(len(y_val))
print(len(predictions))



import seaborn as sns

cm = tf.math.confusion_matrix(labels=y_val, predictions=predictions, num_classes=2)
# Normalize the confusion matrix so that each row sums to 1.
cm = tf.cast(cm, dtype=tf.float32)
cm = cm / tf.math.reduce_sum(cm, axis=1)[:, np.newaxis]

sns_plot = sns.heatmap(cm, annot=True, xticklabels=classes, yticklabels=classes);
plt.xlabel("Predicted");
plt.ylabel("True");

sns_plot.figure.savefig("../confusion_matrix.png")

**VALIDATION PART**

i did the download of the h5 file, from the revision (save version 1)
Upload the file to a new directory and execute the code below.
This step is importante because dont need train the model again...